In [0]:
def import_data(url):
  import pandas as pd
  df = pd.read_csv(url)
  
  # Drop columns not used in model 
    # user_facebook_id, user_profile_url, social_share_last_update - high majority null
    # url, default_url, campaign_image_url, user_profile_url, user_first_name, user_last_name, collected_date - probably won't effect prediction
    # velocity - only 1 value
    # turn_off_donations, deactivated, status, media_type, project_type, category, state, is_launched - majority 1 value
    #, 'user_first_name', 'user_last_name'
  #df.drop(columns=["user_facebook_id", 'status', 'media_type', 'state', 'collected_date', 'is_launched', 'category', 'project_type', 'Unnamed: 0', "user_profile_url","social_share_last_update", "url", 'deactivated', "default_url", 'campaign_image_url', 'user_profile_url', 'velocity', 'turn_off_donations'], inplace=True)

  # Replace nulls in charity_npo_id with 0
  # Replace nulls in charity_name, description with N/A
  df.charity_npo_id.fillna(0, inplace=True)
  df.charity_name.fillna("N/A", inplace=True)
  df.description.fillna("N/A", inplace=True)

  # cast as bool instead of string 
  df.auto_fb_post_mode = df.auto_fb_post_mode.astype(bool)
  df.visible_in_search = df.visible_in_search.astype(bool)
  df.charity_valid = df.charity_valid.astype(bool)

  # take year of created date, drop every row that wasn't created in 2020
  year = df.created_at.str[:4]
  df['ld_year'] = year 
  df.drop(df[df['ld_year'] != '2020'].index, inplace=True)
  df.drop(columns=['ld_year'], inplace=True)

  # get rid of non alpha numeric
  for col in df:
    if df[col].dtype == str:
        df[col].str.replace('\W', '')

  # separate city and state
  city = df.location_city.str.split(',').str[0]
  state = df.location_city.str.split(',').str[1]

  df['location_city'] = city
  df['location_state'] = state   

    # get rid of any campaigns that just have the name corona
  lst1 = []  
  for row in df.iterrows():
    #print(row)
    if ('corona' in row[1][21] or 'Corona' in row[1][21] or 'corona' in str(row[1][22]) or 'Corona' in str(row[1][22])):
        if not ('coronavirus' in row[1][14] or 'Corona Virus' in row[1][14] or 'Coronavirus' in row[1][14] or 'Corona Virus' in row[1][14] or 'Covid-19' in row[1][14] or 'covid-19' in row[1][14] or 'COVID-19' in row[1][14]):
            #lst1.append(row[1][21] + ' ' + row[1][22])
            df.drop([row[0]], inplace=True)

  for col in df:
    if col[0].isdigit():
      nums = ['zero', 'one', 'two','three','four','five','six', 'seven', 'eight', 'nine']
      df.rename(columns={col:nums[int(col[0])] + '_' + col}, inplace=True)

  return df

df = import_data("/content/drive/My Drive/Colab Notebooks/data/campaigns.csv")  
label = ''
df.head()

,Unnamed: 0,url,campaign_id,auto_fb_post_mode,collected_date,category_id,category,currencycode,current_amount,goal,donators,days_active,days_created,title,description,default_url,has_beneficiary,media_type,project_type,turn_off_donations,user_id,user_first_name,user_last_name,user_facebook_id,user_profile_url,visible_in_search,status,deactivated,state,is_launched,campaign_image_url,created_at,launch_date,campaign_hearts,social_share_total,social_share_last_update,location_city,location_country,location_zip,is_charity,charity_valid,charity_npo_id,charity_name,velocity,location_state
349,349,https://www.gofundme.com/f/aiuto-per-le-comuni...,44439552,False,2020-03-20,18,unknown,EUR,705,2000,33,79.0,79.0,Emergenza Covid19 nelle comunità masai Tanzania,"Siamo Cristina e Ilaria, due amiche c...",aiuto-per-le-comunita-masai-della-tanzania,True,3,1.0,0.0,39364012,Cristina,Pedrinzani,NaN,NaN,True,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/44439552...,2020-01-01T12:13:59-06:00,2020-01-01T14:01:30-06:00,32,108,2020-03-18T10:20:37-05:00,NaN,IT,54100 Massa,False,False,0.0,N/A,0,NaN
350,350,https://www.gofundme.com/f/wj9pq4-team-kiki,43814064,False,3/16/2020,2,unknown,USD,250,8800,2,105.0,105.0,Home Repairs -Job Loss due to Corona Virus,"It all started in 2017, the gutters on my 1960...",wj9pq4-team-kiki,False,3,1.0,0.0,43557168,Keira Kiki,Spaulding,NaN,NaN,False,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/43814064...,2020-01-01T16:10:02-06:00,2019-12-02T16:13:55-06:00,0,0,NaN,Fullerton,US,92835,False,False,0.0,N/A,0,CA
353,353,https://www.gofundme.com/f/1uv38jym5c,44478460,False,3/13/2020,11,unknown,USD,0,500,0,3.0,70.0,Coronavirus Covid19 2Seattle Residents Need Help,My wife and I live in king county Washington w...,1uv38jym5c,False,3,1.0,0.0,44213524,Antawn,Emery,2.792160e+15,https://graph.facebook.com/v3.0/27921647275195...,True,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/44478460...,2020-01-03T08:42:44-06:00,2020-03-10T09:32:28-05:00,0,0,NaN,Kent,US,98031,False,False,0.0,N/A,0,WA
354,354,https://www.gofundme.com/f/raising-money-for-a...,44472302,False,3/16/2020,9,unknown,USD,125,1000,2,74.0,74.0,Raising money for a great friend Rolando,Sad news that we announce the passing of Rolan...,raising-money-for-a-seat-friend-rolando,False,3,1.0,0.0,44207472,Griselda,Delgado,9.274070e+14,https://graph.facebook.com/v3.0/92740720063187...,False,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/44472302...,2020-01-04T23:05:37-06:00,2020-01-02T23:14:05-06:00,2,0,NaN,San Jose,US,95112,False,False,0.0,N/A,0,CA
355,355,https://www.gofundme.com/f/help-baby-qianzaiel...,44516468,False,2020-03-23,11,unknown,USD,100,60000,4,79.0,79.0,Help Qiánzài-el Liver Transplant Surgery/ Corona,We’re in dire need of help in raising funds fo...,help-baby-qianzaiel-liver-transplant-surgery,False,3,1.0,0.0,44251166,Lashanda,Gordon,NaN,NaN,False,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/44516468...,2020-01-04T23:25:39-06:00,2020-01-04T23:34:49-06:00,4,0,NaN,Miami,US,33136,False,False,0.0,N/A,0,FL


In [0]:
# Create t-score "success" variable. combination of percent of goal and amount/donation
import numpy as np

#convert everything to USD
df['current_amount_USD'] = 0
df.loc[df['currencycode'] == 'USD', 'current_amount_USD'] = df.current_amount
df.loc[df['currencycode'] == 'EUR', 'current_amount_USD'] = df.current_amount*0.92  #conversion rates at point in time
df.loc[df['currencycode'] == 'GBP', 'current_amount_USD'] = df.current_amount*0.81
df.loc[df['currencycode'] == 'CAD', 'current_amount_USD'] = df.current_amount*1.4
df.loc[df['currencycode'] == 'AUD', 'current_amount_USD'] = df.current_amount*1.62
df.loc[df['currencycode'] == 'CHF', 'current_amount_USD'] = df.current_amount*0.97
df.loc[df['currencycode'] == 'SEK', 'current_amount_USD'] = df.current_amount*10.03
df.loc[df['currencycode'] == 'NOK', 'current_amount_USD'] = df.current_amount*10.24
df.loc[df['currencycode'] == 'DKK', 'current_amount_USD'] = df.current_amount*6.85

df['goal_USD'] = 0
df.loc[df['currencycode'] == 'USD', 'goal_USD'] = df.goal
df.loc[df['currencycode'] == 'EUR', 'goal_USD'] = df.goal*0.92  #conversion rates at point in time
df.loc[df['currencycode'] == 'GBP', 'goal_USD'] = df.goal*0.81
df.loc[df['currencycode'] == 'CAD', 'goal_USD'] = df.goal*1.4
df.loc[df['currencycode'] == 'AUD', 'goal_USD'] = df.goal*1.62
df.loc[df['currencycode'] == 'CHF', 'goal_USD'] = df.goal*0.97
df.loc[df['currencycode'] == 'SEK', 'goal_USD'] = df.goal*10.03
df.loc[df['currencycode'] == 'NOK', 'goal_USD'] = df.goal*10.24
df.loc[df['currencycode'] == 'DKK', 'goal_USD'] = df.goal*6.85

#feature engineering
#percent of goal
df['percent_of_goal'] = df.current_amount / df.goal

#amount per donation
df['amount_per_donation'] = df.current_amount_USD / df.donators
df.loc[df['donators'] == 0, 'amount_per_donation'] = 0
df.loc[df['current_amount_USD'] == 0, 'amount_per_donation'] = 0 

df['percent'] = df.percent_of_goal
df.loc[df['percent'] > 1, 'percent' ] = 1

df['percent_of_goal_cub'] = df['percent']**(1/3)
df.drop(columns='percent', inplace=True)
df['amount_per_donation_lnplus'] = np.log(df['amount_per_donation'] + 1)


# Calculate the mu (mean of the entire population) for each of our four factors:
df['percent_of_goal_score'] = (((df['percent_of_goal_cub']-df['percent_of_goal_cub'].mean())/df['percent_of_goal_cub'].std())*10) + 50
df['amt_per_donor_score'] = (((df['amount_per_donation_lnplus']-df['amount_per_donation_lnplus'].mean())/df['amount_per_donation_lnplus'].std())*10) + 50
df['success_score'] = (df['percent_of_goal_score'] + df['amt_per_donor_score'])/2
print('Percent of goal score: ')
print('Mean: ' + str(df['percent_of_goal_score'].mean()))
print('Min: ' + str(df['percent_of_goal_score'].min()))
print('Median: ' + str(df['percent_of_goal_score'].median()))
print('Max: ' + str(df['percent_of_goal_score'].max()))
print('\nAmount per donor score: ')
print('Mean: ' + str(df['amt_per_donor_score'].mean()))
print('Min: ' + str(df['amt_per_donor_score'].min()))
print('Median: ' + str(df['amt_per_donor_score'].median()))
print('Max: ' + str(df['amt_per_donor_score'].max()))
print('\nSuccess score: ')
print('Mean: ' + str(df['success_score'].mean()))
print('Min: ' + str(df['success_score'].min()))
print('Median: ' + str(df['success_score'].median()))
print('Max: ' + str(df['success_score'].max()))

Percent of goal score: 
Mean: 50.00000000000071
Min: 40.66821940493091
Median: 47.4214792140423
Max: 72.01407472538831

Amount per donor score: 
Mean: 49.999999999999005
Min: 38.20980959720808
Median: 53.373575990103
Max: 89.81491195339873

Success score: 
Mean: 50.00000000000197
Min: 39.439014501069494
Median: 51.495231083653344
Max: 75.74036427825631


In [0]:
df.drop(columns=['amount_per_donation_lnplus', 'percent_of_goal_cub'], inplace=True)
df.head(4)

,Unnamed: 0,url,campaign_id,auto_fb_post_mode,collected_date,category_id,category,currencycode,current_amount,goal,donators,days_active,days_created,title,description,default_url,has_beneficiary,media_type,project_type,turn_off_donations,user_id,user_first_name,user_last_name,user_facebook_id,user_profile_url,visible_in_search,status,deactivated,state,is_launched,campaign_image_url,created_at,launch_date,campaign_hearts,social_share_total,social_share_last_update,location_city,location_country,location_zip,is_charity,charity_valid,charity_npo_id,charity_name,velocity,location_state,current_amount_USD,goal_USD,percent_of_goal,amount_per_donation,percent_of_goal_score,amt_per_donor_score,success_score
349,349,https://www.gofundme.com/f/aiuto-per-le-comuni...,44439552,False,2020-03-20,18,unknown,EUR,705,2000,33,79.0,79.0,Emergenza Covid19 nelle comunità masai Tanzania,"Siamo Cristina e Ilaria, due amiche c...",aiuto-per-le-comunita-masai-della-tanzania,True,3,1.0,0.0,39364012,Cristina,Pedrinzani,NaN,NaN,True,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/44439552...,2020-01-01T12:13:59-06:00,2020-01-01T14:01:30-06:00,32,108,2020-03-18T10:20:37-05:00,NaN,IT,54100 Massa,False,False,0.0,N/A,0,NaN,648.6,1840.0,0.352500,19.654545,62.811051,53.536090,58.173571
350,350,https://www.gofundme.com/f/wj9pq4-team-kiki,43814064,False,3/16/2020,2,unknown,USD,250,8800,2,105.0,105.0,Home Repairs -Job Loss due to Corona Virus,"It all started in 2017, the gutters on my 1960...",wj9pq4-team-kiki,False,3,1.0,0.0,43557168,Keira Kiki,Spaulding,NaN,NaN,False,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/43814064...,2020-01-01T16:10:02-06:00,2019-12-02T16:13:55-06:00,0,0,NaN,Fullerton,US,92835,False,False,0.0,N/A,0,CA,250.0,8800.0,0.028409,125.000000,50.232799,62.689265,56.461032
353,353,https://www.gofundme.com/f/1uv38jym5c,44478460,False,3/13/2020,11,unknown,USD,0,500,0,3.0,70.0,Coronavirus Covid19 2Seattle Residents Need Help,My wife and I live in king county Washington w...,1uv38jym5c,False,3,1.0,0.0,44213524,Antawn,Emery,2.792160e+15,https://graph.facebook.com/v3.0/27921647275195...,True,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/44478460...,2020-01-03T08:42:44-06:00,2020-03-10T09:32:28-05:00,0,0,NaN,Kent,US,98031,False,False,0.0,N/A,0,WA,0.0,500.0,0.000000,0.000000,40.668219,38.209810,39.439015
354,354,https://www.gofundme.com/f/raising-money-for-a...,44472302,False,3/16/2020,9,unknown,USD,125,1000,2,74.0,74.0,Raising money for a great friend Rolando,Sad news that we announce the passing of Rolan...,raising-money-for-a-seat-friend-rolando,False,3,1.0,0.0,44207472,Griselda,Delgado,9.274070e+14,https://graph.facebook.com/v3.0/92740720063187...,False,1,False,active,True,https://d2g8igdw686xgo.cloudfront.net/44472302...,2020-01-04T23:05:37-06:00,2020-01-02T23:14:05-06:00,2,0,NaN,San Jose,US,95112,False,False,0.0,N/A,0,CA,125.0,1000.0,0.125000,62.500000,56.341147,59.220826,57.780986


In [0]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/data/campaignsDB.csv")